In [1]:
import sys  
sys.path.insert(0, '../BendersProblems/')
%matplotlib widget

In [2]:
import pandas as pd
import gurobipy   as      gp
from   gurobipy   import GRB
#from MasterProblem import MasterProblem
#from SubProblem import SubProblem



In [3]:
#path = "C:/Users/willi/OneDrive/Escritorio/Magister/Tesis-Magister/Database/openPitModel/" #Notebook
path = "/home/williams/Tesis-Magister/Databases/"
#path = "C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/ThesisCode/MainCode/Databases/openPitModel/" #Desktop
undergroundDatabaseName = 'Modelo_F_OG.xlsx'
#openPitDatabaseName = 'Modelo_F_OG.xlsx'
openPitDatabaseName = 'modelo_reblogeado.xlsx'

In [4]:
if undergroundDatabaseName == openPitDatabaseName:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = undergroundMineDataframe
else:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = pd.read_excel(path + openPitDatabaseName, engine="openpyxl") #Notebook

In [5]:
import sys  
sys.path.insert(0, '../AuxiliarCodes/')
import gurobipy   as     gp
from   gurobipy   import GRB
from drawpointFunction  import drawpointFunction
from globalFunctions import getNumberOfBlocksInADimension
from itertools import chain
from functools import reduce


class MasterProblem:
    #Underground Model + Crown Pillar Restrictions.
    def __init__(self, database, numberOfPeriods, colHeight, minColHeight,pos_x,pos_y,pos_z,pos_x_f,pos_y_f):
        self.database = database
        self.numberOfPeriods = numberOfPeriods
        self.DP_init = 0       #### Tipo de extracción
        self.desc = 0.1
        self.colHeight = colHeight#630#300
        self.minColHeight = minColHeight#0.20
        self.pos_x = pos_x     
        self.pos_y = pos_y   
        self.pos_z = pos_z
        self.pos_x_f = pos_x_f     
        self.pos_y_f = pos_y_f  
        self.p_t = 3791.912
        self.orientationToExtractTheDrawpoints = 0

    def setParameters(self):
        self.getUndergroundVariablesFromCSV()
        self.getUndergroundInfo()
        self.setUndergroundParameters()
        self.setUndergroundMineLimits()
        self.setUndergroundVariables()
        #self.setVandB_vParameters()
    
    def getUndergroundVariablesFromCSV(self):
        self.undergroundBlocksLenght = self.database['X'].to_dict()             
        self.undergroundBlocksWidth  = self.database['Y'].to_dict()             
        self.undergroundBlocksHeight = self.database['Z'].to_dict()             
        self.undergroundBlockTonnage = self.database['Ton'].to_dict()              
        self.undergroundBlockMineral  = self.database['Mineral'].to_dict()          
        self.undergroundBlockRecovery  = self.database['Recuperación'].to_dict()     
        self.undergroundCopperLaw  = self.database['%Cu'].to_dict()
        self.undergroundExtractionFixedCosts = self.database['CPlanta CA'].to_dict()
        self.undergroundVariableExtractionCosts = self.database['CMina CA'].to_dict()
        self.undergroundCP_S = self.database['CPlanta S'].to_dict()
        self.undergroundCM_S = self.database['CMINA S'].to_dict() 
    
    def getUndergroundInfo(self):
        self.undergroundBlocks = [i for i in range(len(self.undergroundBlocksLenght.values()))]

    def setUndergroundParameters(self):
        #Underground Parameters
        self.t_S   = {period : period + 1 for period in range(self.numberOfPeriods)}
        self.MU_mt = {period : 25806600.0/1  for period in range(self.numberOfPeriods)} #Tonleage es mina
        self.ML_mt = {period : 0/3  for period in range(self.numberOfPeriods)}
        self.MU_pt = {period : 17777880.0/1  for period in range(self.numberOfPeriods)}
        self.ML_pt = {period : 0/3 for period in range(self.numberOfPeriods)}
        self.qU_dt = {period : 1 for period in range(self.numberOfPeriods)}
        self.qL_dt = {period : 0.0001 for period in range(self.numberOfPeriods)}
        self.A_d   = {period : 2 for period in range(self.numberOfPeriods)}
        self.NU_nt = {period : 59 for period in range(self.numberOfPeriods)} 
        self.NL_nt = {period : 0 for period in range(self.numberOfPeriods)}
        self.N_t   = {period : 59* (1 + period) for period in range(self.numberOfPeriods)}
        self.RL_dt = {period : 0.2 for period in range(self.numberOfPeriods)}
        self.RU_dt = {period : 1 for period in range(self.numberOfPeriods)}

    def setUndergroundMineLimits(self):
        self.undergroundBlocksLenghtLimits = getNumberOfBlocksInADimension(self.undergroundBlocksLenght)
        self.undergroundBlocksWidthLimits = getNumberOfBlocksInADimension(self.undergroundBlocksWidth)
        self.undergroundBlocksHeightLimits = getNumberOfBlocksInADimension(self.undergroundBlocksHeight)

    def setUndergroundVariables(self):
        self.drawpoint, self.G_d, self.Q_d,self.q_d, self.C_pdt, self.C_mdt, self.predecessor, self.x_draw,self.y_draw, self.z_draw, self.drawpoints_blocks = drawpointFunction(
                        self.pos_x, self.pos_y, self.pos_z, self.colHeight, self.DP_init, self.undergroundBlocksLenghtLimits, self.undergroundBlocksWidthLimits, self.undergroundBlocksHeightLimits, self.undergroundBlockTonnage, self.undergroundCP_S, self.undergroundCM_S, self.undergroundBlockMineral,
                        self.undergroundCopperLaw, self.pos_x_f, self.pos_y_f,self.orientationToExtractTheDrawpoints)
        self.predecessorDict = {}
        self.predecessorDict[0] = []
        self.predecessorDict[1] = [0]
        for i in range(1,len(self.predecessor)):
            if self.predecessor[i][0] not in self.predecessorDict.keys():
                self.predecessorDict[self.predecessor[i][0]] = []
            self.predecessorDict[self.predecessor[i][0]].append(self.predecessor[i][1])

    def setModelandGetResults(self):
        self.objValue, self.variableValues, self.runtime, self.gap = self.setUndergroundModel()

    def setModel(self):                
        self.undergroundModel = gp.Model(name = 'Modelo Integrado')
        self.undergroundModel.Params.TimeLimit = 3600
        self.undergroundModel.Params.OutputFlag = 0

        # Underground  Model

              #14. Naturaleza de las variables
        self.x_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="x_d")
        self.y_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.CONTINUOUS, name="y_d")
        self.z_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="z_d")

        #1. Restricción sobre la cantidad de tonelaje máxima y mínima a extraer en cada periodo.
        Ton_Up = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti]*self.G_d[d] for d in self.drawpoint) <= self.MU_mt[ti] for ti in self.t_S),
                                            "Min_max")
        
        Ton_low = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] * self.G_d[d] for d in self.drawpoint) >= self.ML_mt[ti] for ti in self.t_S),
                                            "Min_min")
        #2. Restricción sobre la cantidad de material máxima y mínima a procesar en cada periodo.
        Mat_Up = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] * self.Q_d[d] for d in self.drawpoint) <= self.MU_pt[ti] for ti in self.t_S),
                                            "Mat_max")

        Mat_low = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] * self.Q_d[d] for d in self.drawpoint) >= self.ML_pt[ti] for ti in self.t_S)
                                            , "Mat_min")

        
        #3. Rango de leyes máximas y mínimas a procesar
        GQC_low = self.undergroundModel.addConstrs((gp.quicksum(self.Q_d[d] * self.q_d[d] * self.y_dt[d, ti] for d in self.drawpoint) >=
                                self.qL_dt[ti] * gp.quicksum(self.G_d[d] * self.y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_low")
        
        GQC_Up = self.undergroundModel.addConstrs((gp.quicksum(self.Q_d[d] * self.q_d[d] * self.y_dt[d, ti] for d in self.drawpoint) <=
                                self.qU_dt[ti] * gp.quicksum(self.G_d[d] * self.y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_Up")

        #4. Todos los puntos de extracción deben ser iniciados en el largo de la extracción
        Drawp_init = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for ti in self.t_S) == 1 for d in self.drawpoint), "Drawp_init")

        #5. Los puntos de extracción deben ser activados al menos en el mismo periodo para que se inicie la extracción 
        Drawpextract_61 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, tau] for tau in range(ti+1)) >= self.z_dt[d, ti]  
                                            for d in self.drawpoint for ti in self.t_S), "Drawpextract_61")


        #6. Existe una cantidad máxima y mínima de drawpoints a abrir en cada periodo.
        Drawpextract_64_1 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for d in self.drawpoint) <= self.NU_nt[ti] for ti 
                                                        in self.t_S)
                                                        ,"Drawpextract_64_1")
        Drawpextract_64_2 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for d in self.drawpoint) >= self.NL_nt[ti] for ti 
                                                        in self.t_S)
                                                        , "Drawpextract_64_2")

        #7. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        Drawpextract_65 = self.undergroundModel.addConstrs((gp.quicksum(self.z_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")


        #8. Si iniciamos la extracción de un drawpoint esta debe durar por su duraci ́on determinada.
        ## Un drawpoint solamente puede ser extraido por un preiodo pre determinado (A_d)
        Drawpextract_62 = self.undergroundModel.addConstrs((gp.quicksum(self.z_dt[d, ti] for ti in self.t_S)  <= self.A_d[ti]  for d in self.drawpoint
                                                    for ti in self.t_S), "Drawp_62")

        ## Una vez se inicia extrayendo de un drawpoint, se continua extrayendo sin interrupción
        Drawpextract_63 = self.undergroundModel.addConstrs((self.A_d[ti] *(self.z_dt[d, ti] - self.z_dt[d, ti+1]) 
                                            - gp.quicksum(self.z_dt[d, tau] for tau in range(ti+1)) <= 0 
                                            for d in self.drawpoint for ti in range(0,max(self.t_S))), "Drawpextract_63")

        #9. Relación de variables, el porcentaje a extraer es 0 si no se extra un drawpoint.
        Drawpextract_66 = self.undergroundModel.addConstrs((self.y_dt[d, ti] <= self.z_dt[d, ti] for d in self.drawpoint for ti in self.t_S),
                                                    "Drawpextract_66")

        #10. Existe una tasa m ́ınima de extracci ́on para cada drawpoint a extraer.
        Drawpextract_67_1 = self.undergroundModel.addConstrs((self.RL_dt[ti] * self.z_dt[d, ti]  <=  self.y_dt[d, ti] for d in self.drawpoint
                                                        for ti in self.t_S), "Drawpextract_67_1")

        #11. La altura a extraer debe ser mayor a una cantidad m ́ınima.
        rest_11 = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d,ti] for ti in self.t_S)>= self.minColHeight for d in self.drawpoint))

        #12. No podemos extraer más del 100 % de un drawpoint.
        Reserver_cnst = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] for ti in self.t_S) <= 1 for d in self.drawpoint),
                                                    "Reserver_cnst")

        #13. Si se activa un drawpoint, se extrae en ese periodo
        rest_13 = self.undergroundModel.addConstrs(self.x_dt[d,ti] <= self.z_dt[d, ti] for d in self.drawpoint for ti in self.t_S)

        #14. Naturaleza de variables.

        #15. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        rest_15 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")
        
        #16. Restricción sobre el inicio de la extracci ́on de los drawpoints.


        alternative = self.undergroundModel.addConstrs(gp.quicksum(self.x_dt[a,s] for s in range(0,ti+1)) >= self.x_dt[d, ti] for d in self.drawpoint for ti in self.t_S for a in self.predecessorDict[d])
        #resta_prec = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[self.predecessor[l][0], m]*(max(self.t_S)-m+1) for m in self.t_S) <=
        #                            gp.quicksum(self.x_dt[self.predecessor[l][1], m]*(max(self.t_S)-m+1) for m in self.t_S)  
        #                            for l in range(len(self.predecessor))), "DP_Sup")

        
        #Conjuntos para el crown pillar

        #Restricciones del crown pillar
        #Variable 1 si y solo si el crown pillar esta ubicado en la elevaci ́on v, 0 en otro caso.
        test_w = {745: 0.0,
            785: 0.0,
            825: 0.0,
            865: 0.0,
            905: 0.0,
            945: 0.0,
            985: 1.0,
            1025: 0.0,
            1065: 0.0,
            1105: 0.0,
            1145: 0.0,
            1185: 0.0,
            1225: 0.0,
            1265: 0.0,
            1305: 0.0}

        test_u = {0: 0.0,
            1: 1.0,
            2: 1.0,
            3: 1.0,
            4: 1.0,
            5: 1.0,
            6: 1.0,
            7: 1.0,
            8: 1.0,
            9: 1.0}
        
        self.w_v = self.undergroundModel.addVars(self.V, vtype=GRB.BINARY, name="w")
        #self.w_v = self.undergroundModel.addVars(self.V,lb=test_w, ub=test_w ,vtype=GRB.BINARY, name="w")

        
        
        self.theta = self.undergroundModel.addVar(vtype=GRB.CONTINUOUS,name="theta")
        self.u_t = self.undergroundModel.addVars(self.t_S, vtype=GRB.BINARY, name="u")
        #self.u_t = self.undergroundModel.addVars(self.t_S,lb=test_u, ub=test_u ,vtype=GRB.BINARY, name="u")


        ##Código experimental
        #natu = self.undergroundModel.addConstr(gp.quicksum(self.u_t[t] for t in self.t_S) <= 4)
        time_rest = self.undergroundModel.addConstrs(self.u_t[t-1]<= self.u_t[t] for t in range(1, len(self.t_S)))
        limit_time = self.undergroundModel.addConstrs(self.y_dt[d,t] <= self.u_t[t] for d in self.drawpoint for t in self.t_S)

        pillar_2 = self.undergroundModel.addConstrs(gp.quicksum(self.y_dt[d, ti] 
                                                        for ti in self.t_S) <= self.rho_v[v] * self.w_v[v] + (1 - self.w_v[v]) for v in self.V for d in self.drawpoint)
       
        pillar_3 = self.undergroundModel.addConstr(gp.quicksum(self.w_v[v] for v in self.V) == 1)

        theta_restriction_1 = self.undergroundModel.addConstr(-gp.GRB.INFINITY <= self.theta)
        theta_restriction_2 = self.undergroundModel.addConstr(self.theta <= 9999999999)

         #Función objetivo
        self.undergroundObjectiveFunction = self.theta + gp.quicksum(self.y_dt[d, ti]*((((self.p_t * self.q_d[d] - self.C_pdt[d] ) * self.Q_d[d])-(self.C_mdt[d]*self.G_d[d]))/
                                        ((1+self.desc)**(self.t_S[ti]))) for ti in self.t_S for d in self.drawpoint)

        self.undergroundModel.setObjective(self.undergroundObjectiveFunction, GRB.MAXIMIZE)
        self.undergroundModel.Params.MIPGap = 0.01

    def optimize(self):
        self.undergroundModel.optimize()
        #self.lista_variable_Integrado = (self.undergroundModel.getAttr(GRB.Attr.X, self.undergroundModel.getVars()))
        #solucion = self.undergroundModel.objVal
        #runtime = self.undergroundModel.Runtime
        #gap_f = self.undergroundModel.MIPGap
        estimatedW_v = {key:value.X for key,value in self.w_v.items()}
        estimatedU_t = {key:value.X for key,value in self.u_t.items()}
        self.selectedHeight = [key for key,value in estimatedW_v.items() if value > 0][0]
        self.selectedTime = [key for key,value in estimatedU_t.items() if value > 0][0]

        #self.x_dt_values = self.undergroundModel.getAttr('X', self.x_dt)
        #self.y_dt_values = self.undergroundModel.getAttr('X', self.y_dt)
        #self.z_dt_values = self.undergroundModel.getAttr('X', self.z_dt)
        return estimatedW_v, estimatedU_t,self.theta

In [6]:
import gurobipy   as     gp
from   gurobipy   import GRB
from itertools import chain
from globalFunctions import getNumberOfBlocksInADimension
from openPitFunctions import finalBlock


class SubProblem:
   #OpenPit Problem 
   def __init__(self, database, minHeightUnderground, maxHeightUnderground,numberOfPeriods, safetyLevel):
      self.database = database
      self.numberOfPeriods = numberOfPeriods
      self.minHeightUnderground = minHeightUnderground
      self.maxHeightUnderground = maxHeightUnderground
      self.safetyLevel = safetyLevel
      self.numberOfDestinations = 1
      self.basePrice = 3791.912
      self.desc = 0.1

   def setParameters(self):
      self.setOpenPitVariables()
      self.getOpenPitInfo()
      self.setOpenPitParameters()
      self.setOpenPitMineLimits()
      self.setPossibleHeights()
      self.setHeightSets()

   def setOpenPitVariables(self):
      self.openPitBlocksLength = self.database['X'].to_dict() 
      self.openPitBlocksWidth = self.database['Y'].to_dict() 
      self.openPitBlocksHeight = self.database['Z'].to_dict() #Los bloques se orientan de abajo hacia arriba, el bloque 0 es el que esta más abajo, 784 bloques
      self.L_b = self.database['Ton'].to_dict() #openPitBlockTonnage
      self.o_b = self.database['Mineral'].to_dict() #openPitBlockMineral
      self.openPitBlockRecovery = self.database['Recuperación'].to_dict() #openPitBlockRecovery
      self.openPitCopperLaw = self.database['%Cu'].to_dict() #openPitCopperLaw
      self.c_pbt = self.database['CPlanta CA'].to_dict() #openPitPlantCapacity
      self.c_mbt = self.database['CMina CA'].to_dict() #openPitMineCapacity

   def getOpenPitInfo(self):
      self.openPitBlocks = [i for i in range(len(self.openPitBlocksLength.values()))]

   def setOpenPitParameters(self):
      #OpenPit Parameters
      self.t_C   = {period : period + 1 for period in range(self.numberOfPeriods)}
      self.RMu_t = {period : 2*25806600.0/1 for period in range(self.numberOfPeriods)}#Superior infinita, 0 por abajo Originales: 13219200
      self.RMl_t = {period : 0.0/3 for period in range(self.numberOfPeriods)}#Valor original 8812800.0
      self.RPu_t = {period : 2*17777880.0/1 for period in range(self.numberOfPeriods)}#Valor original 10933380.0
      self.RPl_t = {period : 0/3 for period in range(self.numberOfPeriods)}#Valor original 7288920.0 
      self.qu_t  = {period : 1 for period in range(self.numberOfPeriods)}#Leyes promedio maxima y minima.
      self.ql_t  = {period : 0.0001 for period in range(self.numberOfPeriods)}
      self.delta = {period: 0 for period in range(self.numberOfPeriods)}
      self.maxTimeOpenPit = self.t_C[max(self.t_C)]

   def setOpenPitMineLimits(self):
      self.openPitBlocksLengthLimits = getNumberOfBlocksInADimension(self.openPitBlocksLength)
      self.openPitBlocksWidthLimits = getNumberOfBlocksInADimension(self.openPitBlocksWidth)
      self.openPitBlocksHeightLimits = getNumberOfBlocksInADimension(self.openPitBlocksHeight)
      self.predecessorBlock = self.setPredecessorBlocks()

   def setPredecessorBlocks(self):
      self.predecessorBlocks = finalBlock(self.openPitBlocks, self.openPitBlocksLengthLimits,self.openPitBlocksWidthLimits, self.openPitBlocksHeightLimits)
  
   def setPossibleHeights(self):
      self.blockHeight, self.maxHeight, self.minHeight, self.numOfDifferentsBlocks = self.openPitBlocksHeightLimits
   
   def setHeightSets(self):
      #Acá hay que redifinir self.B_v para que tenga las alturas de maxheight al sumarle el safetylvl

      self.V = [height for height in chain(range(self.minHeight,self.maxHeight,self.blockHeight), [self.maxHeight])]
      self.B_v = {}
      self.rho_v = {v:( ((v- self.safetyLevel - self.minHeightUnderground)/(self.maxHeightUnderground - self.minHeightUnderground)) if v - self.minHeightUnderground > 0 else 0 ) for v in self.V}

      for v in self.V:
         numberOfBlocksBelowV = (self.openPitBlocksLengthLimits[3]*self.openPitBlocksWidthLimits[3])*((v-self.minHeight)/self.openPitBlocksHeightLimits[0])
         blocksBelowV = [block for block in range(int(numberOfBlocksBelowV)) if not numberOfBlocksBelowV == 0]
         self.B_v[v] = blocksBelowV

   def setModel(self, isFinalIteration = False):#,w_opt):
      self.openPitModel = gp.Model(name = 'Modelo openPit')
      self.openPitModel.Params.TimeLimit = 3600
      self.openPitModel.Params.OutputFlag = 0
      
      # Open Pit tengo la variable del modelo cielo abierto continua
      self.x_bt = self.openPitModel.addVars(self.t_C, self.openPitBlocks, vtype=GRB.CONTINUOUS, name="x_b")
      
      #1. Restricci ́on sobre la cantidad de tonelaje m ́axima y m ́ınima a extraer en cada periodo.
      openPitModel_Ton_Up  = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.L_b[b] for b in self.openPitBlocks) 
                              <= self.RMu_t[ti] for ti in self.t_C), "Ton_max")
      openPitModel_Ton_low = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.L_b[b] for b in self.openPitBlocks) 
                              >= self.RMl_t[ti] for ti in self.t_C), "Ton_min")

      #2. Restricci ́on sobre la cantidad de material m ́axima y m ́ınima a extraer en cada periodo.
      openPitModel_Mat_Up_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.o_b[b] for b in self.openPitBlocks) <= 
                              self.RPu_t[ti] for ti in self.t_C), "Mat_max")
      openPitModel_Mat_low_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.o_b[b] for b in self.openPitBlocks) >= 
                              self.RPl_t[ti] for ti in self.t_C), "Mat_min")

      #3. Restricci ́on de precedencia de los bloques a extraer, debemos extraer los 5 bloques superiores al bloque objetivo para sacar a este
      
      #BLOCK_SUP_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[s, self.predecessorBlock[l][0]]*(self.maxTimeOpenPit-s+1) for s in self.t_C) <= 
      #                              gp.quicksum(self.x_bt[s, self.predecessorBlock[l][1]]*(self.maxTimeOpenPit-s+1) for s in self.t_C)  
      #                           for l in range(len(self.predecessorBlock))), "Superior_Block")
      openPitModel_Precedence = self.openPitModel.addConstrs(gp.quicksum(self.x_bt[s,a] for s in range(0,ti+1)) >= self.x_bt[ti, b] for b in self.openPitBlocks for ti in self.t_C for a in self.predecessorBlocks[b])


      #4. Restricci ́on sobre la ley m ́axima y m ́ınima por periodo.
      openPitModel_GQC_Up_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.L_b[b]*self.openPitCopperLaw[b] for b in self.openPitBlocks) <=
                           self.qu_t[ti] * gp.quicksum(self.x_bt[ti, b]*self.L_b[b] for b in self.openPitBlocks) for ti in self.t_C), 
                              "openPitModel_GQC_Up")

      openPitModel_GQC_low_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.L_b[b]*self.openPitCopperLaw[b] for b in self.openPitBlocks) >=
                           self.ql_t[ti] * gp.quicksum(self.x_bt[ti, b]*self.L_b[b] for b in self.openPitBlocks) for ti in self.t_C), 
                              "openPitModel_GQC_LOW")

      #5. Podemos extraer el bloque en un solo periodo.
      openPitModel_Reserve_cons_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b] for ti in self.t_C) <= 1 for b in self.openPitBlocks), 
                              "openPitModel_Reserve_cons")

      #Función objetivo
      self.openPitObjectiveFunction = gp.quicksum(self.x_bt[ti, b]*((((self.basePrice*self.openPitCopperLaw[b]-self.c_pbt[b])*self.o_b[b])-(self.c_mbt[b]*self.L_b[b]))/((1+self.desc)**self.t_C[ti]))
                  for ti in self.t_C for b in self.openPitBlocks)
   
      self.openPitModel.setObjective(self.openPitObjectiveFunction, GRB.MAXIMIZE)
      self.openPitModel.Params.MIPGap = 0.01
      
      
      runtime = self.openPitModel.Runtime


   def execute(self, estimatedW_v, estimatedU_t):
      #Ojo que esta restricción quizás no es equivalente a sumar sobre todo0s los bloques y multiuplicar el lado derecho por la cantidad de bloques debajo de B_v
      #     self.heightRestriction = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b] for b in self.B_v[v]) <= (1 - estimatedW_v[v]) for v in self.V for ti in self.t_C), "heightRestriction")
      self.setModel()
      #print(f'W_v estimado: {estimatedW_v}\nU_t estimado: {estimatedU_t}')
      self.heightRestriction = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b] for b in self.B_v[v]) <= (len(self.B_v[v]) * (1 - estimatedW_v[v])) for v in self.V for ti in self.t_C), "heightRestriction")
      #self.heightRestriction = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b] for ti in self.t_C) <= (1 - estimatedW_v[v]) for v in self.V for b in self.B_v[v]), "heightRestriction")

      self.periodRestriction = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b] for b in self.openPitBlocks ) <= (len(self.openPitBlocks) * (1 - estimatedU_t[ti])) for ti in self.t_C), "timeRestriction")

      self.openPitModel.optimize()
      gp.GRB.QCPDual = True


      #objVal = self.openPitModel.objVal

      if self.openPitModel.Status == gp.GRB.OPTIMAL:
         print("La solución del subproblema es óptima")
         self.pi_bDict = {}
         
         for v in self.V:
            for ti in self.t_C:
               self.pi_bDict[v,ti] = self.heightRestriction[v, ti].pi
         """
         for v in self.V:
            for b in self.B_v[v]:
               self.pi_bDict[v,b] = self.heightRestriction[v, b].pi      
         """
         #self.pi_vb = {}
         #for t in self.t_C:
         #   self.pi_vb[t] = sum(self.pi_bDict[v, t] for v in self.V)

         self.sigma_bDict = {}
         for ti in self.t_C:
            self.sigma_bDict[ti] = self.periodRestriction[ti].pi
         
         self.x_bt_values = self.openPitModel.getAttr('X', self.x_bt)
         objVal = self.openPitObjectiveFunction.getValue()
         self.lista_variable_Integrado = self.openPitModel.getAttr(GRB.Attr.X, self.openPitModel.getVars())

      #elif self.openPitModel.Status == gp.GRB.INFEASIBLE:
      else:
         self.pi_v = [0 for dualVariable in self.heightRestriction.values()]
         objVal = 0

         print("El estado de la solución no es óptimo")
      #print(self.pi_bDict)
      #print(self.sigma_bDict)
      return objVal, self.pi_bDict, self.sigma_bDict


In [7]:
class Main:
    def __init__(self, undergroundMineDataframe, openPitDataframe):
        self.openPitDataframe = openPitDataframe
        self.undergroundMineDataframe = undergroundMineDataframe
        self.numberOfPeriods = 5
        self.epsilon = 10
        self.safetyLevel = 60
        self.colHeight = 790#300 max 630 altura total de los drawpoints
        self.minColHeight = 0.40
        self.pos_x = 440#440#Coordenada x desde donde empezamos a extraer     
        self.pos_y = 550#550#Coordenada y desde donde empezamos a extraer
        self.pos_z = 530#780#Coordenada z desde donde empezamos a extraer     
        self.pos_x_f = 720#720#Coordenada x hazta donde extrameos  
        self.pos_y_f = 910#910#Coordenada y hazta donde extrameos
        
    def execute(self):
        self.createModels()
        self.setMasterProblemCrownPillarHeights()
        self.getResults()
    
    def createModels(self):
        self.createSubProblem()
        self.createMasterProblem()

    def createSubProblem(self):
        self.SubProblem = SubProblem(self.openPitDataframe, self.pos_z, self.pos_z + self.colHeight ,self.numberOfPeriods, self.safetyLevel)
        self.SubProblem.setParameters()

    def createMasterProblem(self):
        self.MasterProblem = MasterProblem(self.undergroundMineDataframe, self.numberOfPeriods,self.colHeight,self.minColHeight ,self.pos_x, self.pos_y, self.pos_z, self.pos_x_f, self.pos_y_f)
        self.MasterProblem.setParameters()
        
    def setMasterProblemCrownPillarHeights(self):
        self.MasterProblem.V, self.MasterProblem.rho_v = self.SubProblem.V, self.SubProblem.rho_v
        self.MasterProblem.B_v = self.SubProblem.B_v


    def getResults(self):
        it = 0
        self.MasterProblem.setModel()
        self.SubProblem.setModel()
        currentTheta = -10000000000000
        currentObjVal = -10000000000000
        while True:
            it += 1
            self.estimatedW_v,self.estimatedU_t ,self.estimatedTheta = self.MasterProblem.optimize()
            self.subProblemObjValue, self.pi_vb, self.sigma_t= self.SubProblem.execute(self.estimatedW_v, self.estimatedU_t)
            print("Altura fijada: ", self.MasterProblem.selectedHeight )
            print("Tiempo fijada: ", self.MasterProblem.selectedTime)
            #print("duales:", self.pi_vb)
            newObjVal = self.MasterProblem.undergroundModel.objVal- self.MasterProblem.theta.X + self.subProblemObjValue
            print(it, "Objective Value =", self.MasterProblem.undergroundModel.objVal- self.MasterProblem.theta.X + self.subProblemObjValue)
            print(f'Underground obj value: {self.MasterProblem.undergroundModel.objVal}')
            print("w_v: {}".format(self.MasterProblem.selectedHeight ))
            print("Current theta: {}".format(currentTheta))
            print(f' Underground Objective value: {self.MasterProblem.undergroundModel.objVal-self.MasterProblem.theta.X} \n OpenPit Objective Value {self.subProblemObjValue}\n Current Theta: {self.MasterProblem.theta.X}')

            
            if abs(currentObjVal - newObjVal) <= self.epsilon:
                objVal = self.MasterProblem.undergroundModel.objVal- self.MasterProblem.theta.X + self.subProblemObjValue
                self.subProblemObjValue, self.pi_vb, self.sigma_t= self.SubProblem.execute(self.estimatedW_v, self.estimatedU_t)
                currentTheta = self.estimatedTheta.X
                self.estimatedW_v,self.estimatedU_t ,self.estimatedTheta = self.MasterProblem.optimize()
                print("-----------------------------Valor óptimo encontrado-----------------------------")
                print("Optimal Crown Pillar height: {}".format(self.MasterProblem.selectedHeight ))
                print(self.estimatedW_v)
                print("Optimal ObjVal: {}".format(self.MasterProblem.undergroundModel.objVal))
                print("Optimal Theta: {}".format(self.MasterProblem.theta.X))
                print("Master problem obj val: {}".format(self.MasterProblem.undergroundModel.objVal))
                print("Optimal ObjVal relaxed: {}".format(objVal))
                print(f' Underground Objective value: {self.MasterProblem.undergroundModel.objVal-self.MasterProblem.theta.X} \n OpenPit Objective Value {self.subProblemObjValue}\n Current Theta: {self.MasterProblem.theta.X}')
                #print("Current theta: {}".format(currentTheta))
                break
            
            currentObjVal = self.MasterProblem.undergroundModel.objVal - self.MasterProblem.theta.X + self.subProblemObjValue
            currentTheta = self.estimatedTheta.X

            print("Valor objetivo del subproblema",self.subProblemObjValue)
            #Iteramos sobre todos los bloques y alturas
            #self.MasterProblem.undergroundModel.addConstr(self.MasterProblem.theta <= self.subProblemObjValue + gp.quicksum(gp.quicksum((self.estimatedW_v[v] - self.MasterProblem.w_v[v]) * self.pi_vb[v,b] for b in self.SubProblem.B_v[v]) for v in self.SubProblem.V)+ gp.quicksum(len(self.SubProblem.openPitBlocks) * (self.estimatedU_t[t] - self.MasterProblem.u_t[t]) * self.sigma_t[t] for t in self.MasterProblem.t_S))
            self.MasterProblem.undergroundModel.addConstr(self.MasterProblem.theta <= (self.subProblemObjValue + gp.quicksum(gp.quicksum((self.estimatedW_v[v] - self.MasterProblem.w_v[v]) * len(self.SubProblem.B_v[v]) * self.pi_vb[v,ti] for v in self.SubProblem.V) for ti in self.SubProblem.t_C) + gp.quicksum(len(self.SubProblem.openPitBlocks) * (self.estimatedU_t[t] - self.MasterProblem.u_t[t]) * self.sigma_t[t] for t in self.MasterProblem.t_S)))

            #Juntando en función de t
            #self.MasterProblem.undergroundModel.addConstr(self.MasterProblem.theta <= self.subProblemObjValue + gp.quicksum(gp.quicksum(self.pi_vb[t] * len(self.MasterProblem.B_v[v]) * (self.estimatedW_v[v]-self.MasterProblem.w_v[v]) for t in self.SubProblem.t_C ) for v in self.MasterProblem.V) + gp.quicksum(len(self.SubProblem.openPitBlocks) * (self.estimatedU_t[t] - self.MasterProblem.u_t[t]) * self.sigma_t[t] for t in self.MasterProblem.t_S))
            
            #Iteramos sobre todas las alturas y periodos
            #self.MasterProblem.undergroundModel.addConstr(self.MasterProblem.theta <= (self.subProblemObjValue + gp.quicksum(gp.quicksum((self.estimatedW_v[v] - self.MasterProblem.w_v[v]) * len(self.SubProblem.B_v[v]) * self.pi_vb[v,ti] for ti in self.SubProblem.t_C) for v in self.SubProblem.V) + gp.quicksum(len(self.SubProblem.openPitBlocks) * (self.estimatedU_t[t] - self.MasterProblem.u_t[t]) * self.sigma_t[t] for t in self.MasterProblem.t_S)))


            print()

In [8]:
main = Main(undergroundMineDataframe, openPitDataframe)
main.execute()

3391
Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2023-10-02
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
La solución del subproblema es óptima
Altura fijada:  1290
Tiempo fijada:  0
1 Objective Value = 1427121198.389719
Underground obj value: 11427121197.389719
w_v: 1290
Current theta: -10000000000000
 Underground Objective value: 1427121198.389719 
 OpenPit Objective Value 0.0
 Current Theta: 9999999999.0
Valor objetivo del subproblema 0.0

Set parameter TimeLimit to value 3600
La solución del subproblema es óptima
Altura fijada:  1250
Tiempo fijada:  0
2 Objective Value = 1432206755.3117085
Underground obj value: 11432206754.311708
w_v: 1250
Current theta: 9999999999.0
 Underground Objective value: 1432206755.3117085 
 OpenPit Objective Value 0.0
 Current Theta: 9999999999.0
Valor objetivo 

5 periodos 1290,0, 9 iteraciones 44 segs (1436574164.8307846, 0.0, 3.0517578125e-05)
-----------------------------Valor óptimo encontrado-----------------------------
Optimal Crown Pillar height: 1290
{530: 0.0, 570: 0.0, 610: 0.0, 650: 0.0, 690: 0.0, 730: 0.0, 770: 0.0, 810: 0.0, 850: 0.0, 890: 0.0, 930: 0.0, 970: 0.0, 1010: 0.0, 1050: 0.0, 1090: 0.0, 1130: 0.0, 1170: 0.0, 1210: 0.0, 1250: 0.0, 1290: 1.0}
Optimal ObjVal: 1436574164.830815
Optimal Theta: 3.0517578125e-05
Master problem obj val: 1436574164.830815
Optimal ObjVal relaxed: 1436574164.8307846
 Underground Objective value: 1436574164.8307846 
 OpenPit Objective Value 0.0
 Current Theta: 3.0517578125e-05

10 periodos 930,3 57 iteraciones 16 min 1.6 segs (684587971.3613744, 2332161997.7347503, 2332161997.7348022)
-----------------------------Valor óptimo encontrado-----------------------------
Optimal Crown Pillar height: 930
{530: 0.0, 570: 0.0, 610: 0.0, 650: 0.0, 690: 0.0, 730: 0.0, 770: 0.0, 810: 0.0, 850: 0.0, 890: 0.0, 930: 1.0, 970: -0.0, 1010: -0.0, 1050: -0.0, 1090: -0.0, 1130: -0.0, 1170: -0.0, 1210: -0.0, 1250: -0.0, 1290: -0.0}
Optimal ObjVal: 3016749969.0961766
Optimal Theta: 2332161997.7348022
Master problem obj val: 3016749969.0961766
Optimal ObjVal relaxed: 3016749969.0961246
 Underground Objective value: 684587971.3613744 
 OpenPit Objective Value 2332161997.7347503
 Current Theta: 2332161997.7348022

10 periodos 930,5 53 iteraciones 14 min 45 segs (685946651.5595369, 2332161997.7347503, 2332161997.734791)
-----------------------------Valor óptimo encontrado-----------------------------
Optimal Crown Pillar height: 930
{530: 0.0, 570: 0.0, 610: 0.0, 650: 0.0, 690: 0.0, 730: 0.0, 770: 0.0, 810: 0.0, 850: 0.0, 890: 0.0, 930: 1.0, 970: 0.0, 1010: 0.0, 1050: 0.0, 1090: 0.0, 1130: 0.0, 1170: 0.0, 1210: 0.0, 1250: 0.0, 1290: 0.0}
Optimal ObjVal: 3018108649.2943277
Optimal Theta: 2332161997.734791
Master problem obj val: 3018108649.2943277
Optimal ObjVal relaxed: 3018108649.294287
 Underground Objective value: 685946651.5595369 
 OpenPit Objective Value 2332161997.7347503
 Current Theta: 2332161997.734791

 10 periodos 930,3 57 iteraciones 18 min 19 segs (754074948.3249183, 2309039490.1594954, 2309039490.159485)
 -----------------------------Valor óptimo encontrado-----------------------------
Optimal Crown Pillar height: 930
{530: 0.0, 570: 0.0, 610: 0.0, 650: 0.0, 690: 0.0, 730: 0.0, 770: 0.0, 810: 0.0, 850: 0.0, 890: 0.0, 930: 1.0, 970: 0.0, 1010: 0.0, 1050: 0.0, 1090: 0.0, 1130: 0.0, 1170: 0.0, 1210: 0.0, 1250: 0.0, 1290: 0.0}
Optimal ObjVal: 3063114438.484403
Optimal Theta: 2309039490.159485
Master problem obj val: 3063114438.484403
Optimal ObjVal relaxed: 3063114438.4844136
 Underground Objective value: 754074948.3249183 
 OpenPit Objective Value 2309039490.1594954
 Current Theta: 2309039490.159485



15 periodos 930,4  88 iteraciones 53 min 44 segs (754091079.1129389, 2309039490.1594954, 2309039490.159424)
-----------------------------Valor óptimo encontrado-----------------------------
Optimal Crown Pillar height: 930
{530: 0.0, 570: 0.0, 610: 0.0, 650: 0.0, 690: 0.0, 730: 0.0, 770: 0.0, 810: 0.0, 850: 0.0, 890: 0.0, 930: 1.0, 970: 0.0, 1010: 0.0, 1050: 0.0, 1090: 0.0, 1130: 0.0, 1170: 0.0, 1210: 0.0, 1250: 0.0, 1290: 0.0}
Optimal ObjVal: 3063130569.2723627
Optimal Theta: 2309039490.159424
Master problem obj val: 3063130569.2723627
Optimal ObjVal relaxed: 3063130569.272434
 Underground Objective value: 754091079.1129389 
 OpenPit Objective Value 2309039490.1594954
 Current Theta: 2309039490.159424



In [9]:
len(main.estimatedW_v)

20

In [15]:
a = {530: 0.0, 540: 0.0, 550: 0.0, 560: 0.0, 570: 0.0, 580: 0.0, 590: 0.0, 600: 0.0, 610: 0.0, 620: 0.0, 630: 0.0, 640: 0.0, 650: 0.0, 660: 0.0, 670: 0.0, 680: 0.0, 690: 0.0, 700: 0.0, 710: 0.0, 720: 0.0, 730: 0.0, 740: 0.0, 750: 0.0, 760: 0.0, 770: 0.0, 780: 0.0, 790: 0.0, 800: 0.0, 810: 0.0, 820: 0.0, 830: 0.0, 840: 0.0, 850: 0.0, 860: 0.0, 870: 0.0, 880: 0.0, 890: 0.0, 900: 0.0, 910: 0.0, 920: 0.0, 930: 0.0, 940: 0.0, 950: 0.0, 960: 0.0, 970: 0.0, 980: 0.0, 990: 1.0, 1000: 0.0, 1010: 0.0, 1020: 0.0, 1030: 0.0, 1040: 0.0, 1050: 0.0, 1060: 0.0, 1070: 0.0, 1080: 0.0, 1090: 0.0, 1100: 0.0, 1110: 0.0, 1120: 0.0, 1130: 0.0, 1140: 0.0, 1150: 0.0, 1160: 0.0, 1170: 0.0, 1180: 0.0, 1190: 0.0, 1200: 0.0, 1210: 0.0, 1220: 0.0, 1230: 0.0, 1240: 0.0, 1250: 0.0, 1260: 0.0, 1270: 0.0, 1280: 0.0, 1290: 0.0, 1300: 0.0, 1310: 0.0, 1320: 0.0}
len(a)

80

In [14]:
len(main.MasterProblem.drawpoint)

168

In [10]:
(main.MasterProblem.undergroundModel.objVal-main.MasterProblem.theta.X , main.subProblemObjValue, main.MasterProblem.theta.X)

(1436636191.6704428, 0.0, 3.0517578125e-05)

15 periodos, optimo 930,4
20 iteraciones
 Underground Objective value: 754091079.1129389 
 OpenPit Objective Value 2309039490.1594954
47 min 27 segs de ejecución

10 periodos 6 min 48.6 segs, (684587971.3613744, 2332161997.7347503, 2332161997.7348022)


Optimal Crown Pillar height: 930
{530: 0.0, 570: 0.0, 610: 0.0, 650: 0.0, 690: 0.0, 730: 0.0, 770: 0.0, 810: 0.0, 850: 0.0, 890: 0.0, 930: 1.0, 970: -0.0, 1010: -0.0, 1050: -0.0, 1090: -0.0, 1130: -0.0, 1170: -0.0, 1210: -0.0, 1250: 0.0, 1290: -0.0}
Optimal ObjVal: 2277240901.397484
Optimal Theta: 1523930041.9959717
Master problem obj val: 2277240901.397484
Optimal ObjVal relaxed: 2277240901.397359
 Underground Objective value: 753310859.4015121 
 OpenPit Objective Value 1523930041.9958472
 Current Theta: 1523930041.9959717


Optimal ObjVal TopoSort: 2265301763.1929398


In [11]:
main.estimatedU_t

{0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0}

In [12]:
main.estimatedW_v

{530: 0.0,
 570: 0.0,
 610: 0.0,
 650: 0.0,
 690: 0.0,
 730: 0.0,
 770: 0.0,
 810: 0.0,
 850: 0.0,
 890: 0.0,
 930: 0.0,
 970: 0.0,
 1010: 0.0,
 1050: 0.0,
 1090: 0.0,
 1130: 0.0,
 1170: 0.0,
 1210: 0.0,
 1250: 0.0,
 1290: 1.0}

In [13]:
print(main.MasterProblem.undergroundModel.objVal-main.MasterProblem.theta.X , main.subProblemObjValue, main.MasterProblem.theta.X)

1436636191.6704428 0.0 3.0517578125e-05
